# Package Install
```
$ sudo apt-get install python2.7-dev
$ sudo easy_install pip
$ pip install jupyter
$ pip install pandas
$ pip install google-api-python-client
$ pip install python-gflags
```

# Jupyter spark enviroment.
```
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook"
pyspark --master local[2] --packages graphframes:graphframes:0.3.0-spark1.6-s_2.10
```

In [1]:
import pandas as pd

projectId = 'spark-151209'
query = """SELECT Actor1CountryCode, Actor2CountryCode, EventRootCode FROM [gdelt-bq:full.events] 
WHERE Actor1CountryCode != ""
AND Actor2CountryCode != ""
AND EventRootCode = '19'
LIMIT 1000"""
df = pd.read_gbq(query, projectId)

/Users/rchen/.virtualenvs/spark/lib/python2.7/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access bigquery_credentials.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Failed to start a local webserver listening on either port 8080
or port 8090. Please check your firewall settings and locally
running programs that may be blocking or using those ports.

Falling back to --noauth_local_webserver and continuing with
authorization.


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/v2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&access_type=offline

Enter verification code: 4/rvuK4s7h6dvM4VVJMfhheTfQNsGSkl1Y-BNPjbZ0XIA
Authentication successful.
Requesting query... ok.
Query running...
Query done.
Processed: 3.7 Gb

Retrieving results...
Got 1000 rows.

Total time taken 3.09 s.
Finished at 2017-01-12 11:43:08.


In [2]:
sqlctx = SQLContext(sc)
sdf = sqlctx.createDataFrame(df)
sdf.show()

NameError: name 'SQLContext' is not defined

In [16]:
import graphframes

keys = sdf.flatMap(lambda x: (x[0], x[1])).distinct()
keylist = keys.collect()
vertices = keys.map(lambda x: (x,)).toDF(["id"])
edge = sdf.map(lambda x: (x[0], x[1])).toDF(["src", "dst"])
g = graphframes.GraphFrame(vertices, edge)

In [17]:
results = g.pageRank(resetProbability=0.0001, maxIter=20)


In [18]:
results.vertices.sort('pagerank', ascending=False).select("id", "pagerank").show()

+---+--------------------+
| id|            pagerank|
+---+--------------------+
|AFG|0.010078213182363351|
|USA|0.009731350299274757|
|DEU|0.007294977300554158|
|UGA|0.005693943979311197|
|ISR|0.004657865118146054|
|AUS|0.004073478420682504|
|PSE|0.003365360237612...|
|BGD|0.002944806256266398|
|SYR|0.002818097548021146|
|FRA|0.002690311636449911|
|SAU|0.002136359804535...|
|TTO|0.002097901329401704|
|BDI|0.002097901329401704|
|SLV|0.002097901329401704|
|COL|0.002097901329401704|
|HND|0.002097901329401704|
|AFR| 0.00209609769884462|
|GBR|0.002023427248577...|
|ZAF|0.002000560474343...|
|CAN|0.001753028031950...|
+---+--------------------+
only showing top 20 rows

